# Importing needed packages

In [36]:
import re
import unicodedata
import pprint

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [47]:
pp = pprint.PrettyPrinter(indent=1, width=140, compact=True)

# Parsing main page

## Getting the code for the page with movie

In [2]:
# f = requests.get("https://www.kinopoisk.ru/film/339/")

## Getting the code from the downloaded page

In [3]:
path = "./data/indiana_front.html"
with open(path, 'r', encoding="utf-8") as f:
    indiana_front = f.read()

Passing the code into the `BeautifulSoup` constructor

In [4]:
soup = BeautifulSoup(indiana_front, "html.parser")

In [5]:
movie_dict = {}

## Parsing movie's name

In [6]:
movie_name_dict = {}

In [7]:
movie_name = soup.find_all("h1", attrs={"class": re.compile("styles_title")})[0]

In [8]:
movie_name.get_text()

'Индиана Джонс: В поисках утраченного ковчега (1981)'

In [9]:
movie_name_original = soup.find_all(
    "span", attrs={"class": re.compile("styles_originalTitle")}
)[0]

In [10]:
movie_name_original.get_text()

'Raiders of the Lost Ark'

In [11]:
movie_name_dict["Название"] = movie_name.get_text()
movie_name_dict["Оргинальное название"] = movie_name_original.get_text()

## Parsing information about the movie

In [12]:
movie_info_dict = {}

In [13]:
movie_info_divs = soup.find_all(attrs={"data-test-id": "encyclopedic-table"})[0]

In [14]:
values = []
for div in movie_info_divs.find_all(
    "div", attrs={"class": re.compile("styles_value"), "data-tid": re.compile(".*")}
):
    current_value = div.get_text().replace("сборы", "")
    current_value = unicodedata.normalize("NFKD", current_value)
    if "слова" not in current_value:
        values.append(current_value)

In [15]:
titles = []
for div in movie_info_divs.find_all(class_=re.compile("styles_title")):
    current_title = div.get_text()
    titles.append(current_title)

In [16]:
assert len(values) == len(titles)
movie_info_dict = dict(zip(titles, values))
movie_dict = {**movie_name_dict, **movie_info_dict}

In [17]:
movie_dict

{'Название': 'Индиана Джонс: В поисках утраченного ковчега (1981)',
 'Оргинальное название': 'Raiders of the Lost Ark',
 'Год производства': '1981',
 'Страна': 'США',
 'Жанр': 'приключения, боевик, комедия',
 'Слоган': '«Indiana Jones - the new hero from the creators of JAWS and STAR WARS»',
 'Режиссер': 'Стивен Спилберг',
 'Сценарий': 'Лоуренс Кэздан, Джордж Лукас, Филип Кауфман',
 'Продюсер': 'Ховард Дж. Казанян, Джордж Лукас, Фрэнк Маршалл, ...',
 'Оператор': 'Дуглас Слоком',
 'Композитор': 'Джон Уильямс',
 'Художник': 'Норман Рейнольдс, Айвор Лесли Дилли, Дебора Нэдулмэн, ...',
 'Монтаж': 'Майкл Кан, Джордж Лукас',
 'Бюджет': '$18 000 000',
 'Сборы в США': '$212 222 025',
 'Сборы в мире': '+ $141 766 000 = $353 988 025',
 'Зрители': '88.1 млн , 6.4 млн , 4.2 млн , ...',
 'Премьера в мире': '12 июня 1981, ...',
 'Релиз на DVD': '20 ноября 2003, «Premier Digital»',
 'Возраст': '12+',
 'Рейтинг MPAA': 'PGрекомендуется присутствие родителей',
 'Время': '115 мин. / 01:55'}

## Parsing movie's rating

In [18]:
movie_rating_dict = {}

In [19]:
movie_rating_kinopoisk = soup.find_all("a", attrs={"class": re.compile("film-rating-value")})[0]

In [20]:
movie_rating_kinopoisk.get_text()

'8.0'

In [21]:
movie_rating_count_kinopoisk = soup.find_all("span", attrs={"class": re.compile("styles_count")})[0]

In [22]:
movie_rating_count_kinopoisk.get_text()

'142 538 оценок'

In [23]:
movie_rating_imdb = soup.find_all(
    "span", attrs={"class": re.compile("styles_valueSection")}
)[0]

In [24]:
movie_rating_imdb.get_text()

'IMDb: 8.40'

In [25]:
movie_rating_count_imdb = soup.find_all(
    "div", attrs={"class": re.compile("film-sub-rating")}
)[0].find(
    "span", attrs={"class": re.compile("styles_count")}
)

In [26]:
movie_rating_count_imdb.get_text()

'938 581 оценка'

In [27]:
movie_rating_dict["Рейтинг кинопоиска"] = movie_rating_kinopoisk.get_text()
movie_rating_dict["Количество оценок кинопоиска"] = movie_rating_count_kinopoisk.get_text()
movie_rating_dict["Рейтинг IMDb"] = movie_rating_imdb.get_text()
movie_rating_dict["Количество оценок IMDb"] = movie_rating_count_imdb.get_text()

In [28]:
movie_rating_dict

{'Рейтинг кинопоиска': '8.0',
 'Количество оценок кинопоиска': '142 538 оценок',
 'Рейтинг IMDb': 'IMDb: 8.40',
 'Количество оценок IMDb': '938 581 оценка'}

# Preparing final dict with information about film

In [48]:
movie_dict = {**movie_name_dict, **movie_info_dict, **movie_rating_dict}
pp.pprint(movie_dict)

{'Бюджет': '$18 000 000',
 'Возраст': '12+',
 'Время': '115 мин. / 01:55',
 'Год производства': '1981',
 'Жанр': 'приключения, боевик, комедия',
 'Зрители': '88.1 млн , 6.4 млн , 4.2 млн , ...',
 'Количество оценок IMDb': '938 581 оценка',
 'Количество оценок кинопоиска': '142 538 оценок',
 'Композитор': 'Джон Уильямс',
 'Монтаж': 'Майкл Кан, Джордж Лукас',
 'Название': 'Индиана Джонс: В поисках утраченного ковчега (1981)',
 'Оператор': 'Дуглас Слоком',
 'Оргинальное название': 'Raiders of the Lost Ark',
 'Премьера в мире': '12 июня 1981, ...',
 'Продюсер': 'Ховард Дж. Казанян, Джордж Лукас, Фрэнк Маршалл, ...',
 'Режиссер': 'Стивен Спилберг',
 'Рейтинг IMDb': 'IMDb: 8.40',
 'Рейтинг MPAA': 'PGрекомендуется присутствие родителей',
 'Рейтинг кинопоиска': '8.0',
 'Релиз на DVD': '20 ноября 2003, «Premier Digital»',
 'Сборы в США': '$212 222 025',
 'Сборы в мире': '+ $141 766 000 = $353 988 025',
 'Слоган': '«Indiana Jones - the new hero from the creators of JAWS and STAR WARS»',
 'Стра